# EPDx and Ökobau 

In [12]:
# Import libraries that we need
import json
from pathlib import Path

import epdx
import requests

## Ökobau URL
First we need to define the endpoint from where we can get the EPD data. In this case, we will use [Ökobau](https://www.oekobaudat.de/en.html)

In [14]:
OKOBAU_URL = "https://oekobaudat.de/OEKOBAU.DAT/resource/datastocks/cd2bda71-760b-4fcc-8a0b-3877c10000a8"

In [3]:
def get_epds(limit=10) -> dict:
    """Get EPDs from Ökobau"""

    response = requests.get(f"{OKOBAU_URL}/processes?format=json&pageSize={limit}")
    response.raise_for_status()
    _data = response.json()

    print(f"Retrieved {_data.get('pageSize')} EPDs out of {_data.get('totalCount')} from Ökobau")

    return _data

def parse_epd_list(_data: dict):
    """Prints out the name and UUID of each EPD"""

    for _epd in _data.get("data"):
        print(f"Name: {_epd.get('name')}\tUUID: {_epd.get('uuid')}")


# Let's get some EPDs!
Now we are ready to fetch some EPDs from Ökobau.\
We start by calling the API to get the first 10 EPDs in the database.

For each of the EPDs we will print the name and id.

In [5]:
# Lists EPDs in the Ökobau database

data = get_epds()
parse_epd_list(data)


Retrieved 10 EPDs out of 2285 from Ökobau
Name: 2-Schicht-Parkett	UUID: 2eb43850-0ab2-4068-afe5-218d69a096f8
Name: 3- und 5-Schicht Massivholzplatte (Durchschnitt DE)	UUID: f63ac879-fa7d-4f91-813e-e816cbdf1927
Name: 3- und 5-Schicht Massivholzplatte (Durchschnitt DE)	UUID: 8be9edb5-c5b9-4be1-bfb8-b096f24a183b
Name: 3-Schicht-Parkett	UUID: c8835af8-44d2-47df-a833-8e823e6ee5cd
Name: A2-Betonpflaster- Standardstein grau mit Vorsatz	UUID: 0fdca580-3027-493d-a3c7-815758bac55d
Name: AAC-Block und AAC-Blöcke für Bodenplatten	UUID: b78239d5-d2c5-4794-9112-a0c7907dea72
Name: Abschlüsse - clauss markisen Projekt GmbH - Feuerschutzvorhang	UUID: ee8863aa-7276-4896-b07a-713937a3134d
Name: Abschlüsse - clauss markisen Projekt GmbH - Rauchschutzvorhang	UUID: 4cd68b8d-93e3-4002-8e25-7950350a6c26
Name: Abstandhalter - Ensinger GmbH - Thermix Low Psi	UUID: 7d08d67b-f4e2-4a53-9672-f3b498bfd045
Name: Abstandhalter - Ensinger GmbH - Thermix Low Psi RE	UUID: fc1cd6bc-43e7-467b-a614-65474528a266


In [6]:
def get_full_epd(uid: str) -> dict:
    """Get the full dataset for a single EPD"""

    base_url = f"{OKOBAU_URL}/processes/{uid}"
    response = requests.get(f"{base_url}?format=json&view=extended")

    response.raise_for_status()
    data = response.json()
    data["source"] = base_url

    return data

# Let's Convert to EPDx!

To get the full dataset of an EPD we need to make an additional call to the API.
The response we parse with EPDx, to get a more workable data format.

EPDx makes it easier to extract values from the EPD such as the GWP value.

In [13]:
# Get EPDs from Ökobau and turn them into EPDx

data = get_epds(limit=1)

for _epd in data.get("data"):
    print(f"\nEPD {_epd.get('uuid')}")

    # Get full EPD from Ökobau
    epd = get_full_epd(_epd.get("uuid"))

    # Turn EPD into EPDx dict
    epdx_dict = epdx.convert_ilcd(epd)
    print('EPDx Dict')
    print(json.dumps(epdx_dict, indent=2))

    # Turn EPD into an EPDx Pydantic Class
    epdx_pydantic = epdx.convert_ilcd(epd, as_type=epdx.EPD)
    print('\nEPDx Pydantic')
    print(epdx_pydantic)
    print("---------\n")

Retrieved 1 EPDs out of 2285 from Ökobau

EPD 2eb43850-0ab2-4068-afe5-218d69a096f8
EPDx Dict
{
  "id": "2eb43850-0ab2-4068-afe5-218d69a096f8",
  "name": "2-Schicht-Parkett",
  "declared_unit": "M2",
  "version": "1.1",
  "published_date": 1640995200,
  "valid_until": 1798761600,
  "format_version": "1.1.5",
  "source": null,
  "reference_service_life": null,
  "standard": "EN15804A2",
  "comment": null,
  "location": "RER",
  "subtype": "Generic",
  "conversions": [
    {
      "value": 0.0135,
      "to": "M",
      "meta_data": "{\"name\":\"layer thickness\",\"value\":\"0.0135\",\"unit\":\"m\",\"unitDescription\":\"meters\"}"
    },
    {
      "value": 7.7,
      "to": "KG",
      "meta_data": "{\"name\":\"grammage\",\"value\":\"7.7\",\"unit\":\"kg/m^2\",\"unitDescription\":\"kilograms per square meter\"}"
    }
  ],
  "gwp": {
    "a1a3": 6.529,
    "a4": null,
    "a5": 0.2576,
    "b1": null,
    "b2": 7.037,
    "b3": null,
    "b4": null,
    "b5": 1.462,
    "b6": null,
    "b